In [6]:
import requests
from bs4 import BeautifulSoup as bs
import re
import csv
from multiprocessing import Pool
headers = {'Referer':'http://www.aozora.gr.jp/',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
proxies = {'http': 'http://210.13.77.66:8080','https': 'http://210.13.77.66:8080'}

## 数据储存模块

In [ ]:
import sqlite3

def get_con(func):
    data_path = r"sources2.db"

    def sql_exc():
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = func(cur)
        cur.close()
        conn.commit()
        conn.close()
        return data

    return sql_exc


@get_con
def get_sakuhinn(cur):
    # 查询sakuhinn
    print("正在查询作品列表")
    cur.execute("SELECT * FROM sakuhinndata")
    rows = cur.fetchall()
    return rows

@get_con
def get_sakusya(cur):
    # 查询sakusya
    print("正在查询作者列表")
    cur.execute("SELECT * FROM sakusyadata")
    rows = cur.fetchall()
    return rows

@get_con
def sakuhinntable(cur):
    cur.execute('''CREATE TABLE SAKUHINNDATA 
              (
                SAKUHINNID INTEGER PRIMARY KEY autoincrement, SAKUHINNCATALOG CHAR(2) NOT NULL, SAKUHINNNAME TEXT NOT NULL, SAKUSYA CHAR NULL, SAKUHINNLINK CHAR(60) NULL
              )''')
"""sakuhinnid,sakuhinncatalog,sakuhinnname,sakusya,sakuyhinnlink"""

@get_con
def sakusyatable(cur):
    cur.execute('''CREATE TABLE SAKUSYADATA 
              (
                SAKUSYAID INTEGER PRIMARY KEY autoincrement, SAKUSYACATALOG CHAR(2) NOT NULL, SAKUSYANAME TEXT NOT NULL, SAKUHINN CHAR NULL, SAKUSYALINK CHAR(60) NULL
              )''')

def bulid_db():
    """数据库初始化在此"""
    print("数据库初始化")
    try:
        print("正在创建作品列表。。")
        sakuhinntable()
        print("成功！")
    except Exception as e:
        print(e)
        print('已有work表或创建失败！')
        
    try:
        print("正在创建作者列表。。")
        sakusyatable()
        print("成功！")
    except Exception as e:
        print(e)
        print('已有SAKUSYA表或创建失败！')
        
def sakuhinninsert(_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuhinnlink):
    """插入一行sakuhinn数据"""
    try:
        print("正在插入作品列表： " + _sakuhinnname)
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuhinnlink)
        ins = 'INSERT INTO sakuhinndata (sakuhinncatalog,sakuhinnname,sakusya,sakuhinnlink) VALUES(?, ?, ?, ?)'
        cur.execute(ins, data)

    except Exception as e:
        print(e)
        print('eroo sakuhinninsert')
    finally:
        cur.close()
        conn.commit()
        conn.close()
        
def sakusyainsert(_sakusyacatalog,_sakusyaname,_sakuhinn,_sakusyalink):
    """插入一行sakusya数据"""
    try:
        print("正在插入作者列表： " + _sakusyaname)
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_sakusyacatalog,_sakusyaname,_sakuhinn,_sakusyalink)
        ins = 'INSERT INTO sakusyadata (sakusyacatalog,sakusyaname,sakuhinn,sakusyalink) VALUES(?, ?, ?, ?)'
        cur.execute(ins, data)

    except Exception as e:
        print(e)
        print('eroo sakusyainsert')
    finally:
        cur.close()
        conn.commit()
        conn.close()
        

def get_onesakuhinn(_name):
    """获取某一作品数据"""
    try:
        print("正在查询作品信息： " + _name)
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_name,)
        ins = 'select * from sakuhinndata where sakuhinnname = ?'
        cur.execute(ins, data)
        rows = cur.fetchall()
        cur.close()
        conn.commit()
        conn.close()
        return rows
    except Exception as e:
        print(e)
        cur.close()
        conn.commit()
        conn.close()
        return False
    
def get_onesakusya(_name):
    """获取某一作者数据"""
    try:
        print("正在查询作者信息： " + _name)
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_name,)
        ins = 'select * from sakusyadata where sakusyaname = ?'
        cur.execute(ins, data)
        rows = cur.fetchall()
        cur.close()
        conn.commit()
        conn.close()
        return rows
    
    except Exception as e:
        print(e)
        cur.close()
        conn.commit()
        conn.close()
        return False
    
def get_kanasakuhinn(_sakuhinncatalog):
    """获取某一作品假名类数据"""
    try:
        print("正在获取 " + _sakuhinncatalog + "类作品列表")
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_sakuhinncatalog,)
        ins = 'select * from sakuhinndata where sakuhinncatalog = ?'
        cur.execute(ins, data)
        rows = cur.fetchall()
        cur.close()
        conn.commit()
        conn.close()
        return rows
    except Exception as e:
        print(e)
        cur.close()
        conn.commit()
        conn.close()
        return False
    
def get_kanasakusya(_sakusyacatalog):
    """获取某一作者假名类数据"""
    try:
        print("正在获取 " + _sakusyacatalog + "类作者列表")
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_sakusyacatalog,)
        ins = 'select * from sakusyadata where sakusyacatalog = ?'
        cur.execute(ins, data)
        rows = cur.fetchall()
        cur.close()
        conn.commit()
        conn.close()
        return rows
    
    except Exception as e:
        print(e)
        cur.close()
        conn.commit()
        conn.close()
        return False

def update_sakuhinn(_name,_link):
    # 更新users数据，改model状态时用
    try:
        data_path = r"sources2.db"
        conn = sqlite3.connect(data_path, check_same_thread = False)
        conn.text_factory = str
        cur = conn.cursor()
        data = (_name,_link)
        print("正在更新作品列表数据库： "+data[0]+ ' '+data[1])
        ins = 'update sakuhinndata set sakuhinnlink = ? where sakuhinnname = ?'
        cur.execute(ins, (data[1],data[0]))


    except Exception as e:
        print(e)
        print('eroo update_users')
    finally:
        cur.close()
        conn.commit()
        conn.close()

In [27]:
for data in get_sakuhinn():
    name = data[2]
    link = data[-1].replace('../','')
    update_sakuhinn(name,link)
    print(get_onesakuhinn(name)[-1])

正在查询作品列表
正在查询作品信息： ああ華族様だよ　と私は嘘を吐くのであった
正在查询作品信息： ア、秋
正在查询作品信息： ああ玉杯に花うけて
正在查询作品信息： アーサー王物語
正在查询作品信息： 「ああしんど」
正在查询作品信息： ああ東京は食い倒れ
正在查询作品信息： アーニイ・パイルの前に立ちて
正在查询作品信息： あゝ二十年
正在查询作品信息： Ｒ漁場と都の酒場で
正在查询作品信息： 愛
正在查询作品信息： I Am Not A Poet, But I Am A Poet.
正在查询作品信息： 藍色の蟇
正在查询作品信息： Ｉ駅の一夜
正在查询作品信息： 「愛怨峡」における映画的表現の問題
正在查询作品信息： 哀音
正在查询作品信息： 哀歌
正在查询作品信息： 藍瓶
正在查询作品信息： 相川おけさ
正在查询作品信息： I can speak
正在查询作品信息： 愛卿伝
正在查询作品信息： 愛国歌小観
正在查询作品信息： 『愛国心』私はこう思う
正在查询作品信息： 愛国百人一首に関連して
正在查询作品信息： 愛国百人一首評釈
正在查询作品信息： 愛護若
正在查询作品信息： 愛妻家の一例
正在查询作品信息： 哀詞序
正在查询作品信息： 哀詩数篇
正在查询作品信息： 逢状
正在查询作品信息： 愛書癖
正在查询作品信息： 愛人と厭人
正在查询作品信息： 合図の旗
正在查询作品信息： 愛する人達
正在查询作品信息： 愛読作家についての断片
正在查询作品信息： 愛読した本と作家から
正在查询作品信息： 愛読書の印象
正在查询作品信息： 愛と婚姻
正在查询作品信息： 「愛と死」
正在查询作品信息： 愛と認識との出発
正在查询作品信息： 愛と美について
正在查询作品信息： 愛と平和を理想とする人間生活
正在查询作品信息： 愛に就ての問題
正在查询作品信息： アイヌ語学
正在查询作品信息： アイヌ語のおもしろさ
正在查询作品信息： アイヌ宗教成立の史的背景
正在查询作品信息： アイヌ神謡集
正在查询作品信息： アイヌ族の俚謡
正在查询作品信息： 愛のごとく
正在查询作品信息： 愛の詩集
正在查询作品信息： 青白き公園
正在查询作品信息： 蒼白き巣窟
正在查询作品信息： 青白き夢
正在查询作品信息： 「青白き夢」序
正在查询作品信息： 青水仙、赤水仙
正在查询作品信息： 

正在查询作品信息： 学者アラムハラドの見た着物
正在查询作品信息： 学者安心論
正在查询作品信息： 学者と名誉
正在查询作品信息： 各人心宮内の秘宮
正在查询作品信息： 「学生警鐘」と風
正在查询作品信息： 学生時代
正在查询作品信息： 学生時代の菊池寛
正在查询作品信息： 学生と教養
正在查询作品信息： 学生と生活
正在查询作品信息： 学生と先哲
正在查询作品信息： 学生と読書
正在查询作品信息： 学生の技能と勤労大衆
正在查询作品信息： 楽聖物語
正在查询作品信息： 楽隊
正在查询作品信息： 拡大されゆく国道前線
正在查询作品信息： 咢堂小論
正在查询作品信息： 過去世
正在查询作品信息： 鵞湖仙人
正在查询作品信息： 籠釣瓶
正在查询作品信息： 過去となつたアイルランド文学
正在查询作品信息： 傘
正在查询作品信息： 挿頭花
正在查询作品信息： 累物語
正在查询作品信息： 風見
正在查询作品信息： 火山の名について
正在查询作品信息： 火山の話
正在查询作品信息： 火事
正在查询作品信息： 鰍沢雪の夜噺（小室山の御封、玉子酒、熊の膏薬）
正在查询作品信息： 鰍の卵について
正在查询作品信息： 火事教育
正在查询作品信息： 賢い秀雄さんの話
正在查询作品信息： カシタンカ
正在查询作品信息： 佳日
正在查询作品信息： 夏日小味
正在查询作品信息： 画室談義
正在查询作品信息： 画室の言葉
正在查询作品信息： 火事とポチ
正在查询作品信息： 鍛冶の母
正在查询作品信息： 菓子の譜
正在查询作品信息： 「樫の芽生え」を読みて
正在查询作品信息： 貸間を探がしたとき
正在查询作品信息： 貸家探し
正在查询作品信息： 鍛冶屋の子
正在查询作品信息： 貸家を探す話
正在查询作品信息： 歌集『仰日』の著者に
正在查询作品信息： 歌集『集団行進』に寄せて
正在查询作品信息： 歌集「嘲笑」序文
正在查询作品信息： 夏秋表
正在查询作品信息： 歌集『涌井』を読む
正在查询作品信息： 家常茶飯　附・現代思想
正在查询作品信息： 過剰の意識
正在查询作品信息： “歌笑”文化
正在查询作品信息： 花燭
正在查询作品信息： かしはばやしの夜
正在查询作品信息： かしわばやしの夜
正在查询作品信息： 潮霧
正在查询作品信息： カズイスチカ
正在查询作品信息： かすかな声
正在查询作品信息：

正在查询作品信息： 大礼服の例外的効果
正在查询作品信息： 対話
正在查询作品信息： 対話させる術
正在查询作品信息： 台湾の姿態
正在查询作品信息： 下町
正在查询作品信息： 高い木とからす
正在查询作品信息： 他界に対する観念
正在查询作品信息： 高尾紀行
正在查询作品信息： 高尾ざんげ
正在查询作品信息： 高島異誌
正在查询作品信息： 高瀬舟
正在查询作品信息： 竹の里人　一
正在查询作品信息： 竹の里人〔一〕
正在查询作品信息： 竹の里人〔三〕
正在查询作品信息： 竹の里人〔二〕
正在查询作品信息： 竹の根の先を掘るひと
正在查询作品信息： 竹本綾之助
正在查询作品信息： 竹藪の家
正在查询作品信息： 茸をたずねる
正在查询作品信息： 章魚人夫
正在查询作品信息： 章魚の足
正在查询作品信息： 章魚木の下で
正在查询作品信息： 蛸の如きもの
正在查询作品信息： 凧の話
正在查询作品信息： ダゴン
正在查询作品信息： 太宰治情死考
正在查询作品信息： 太宰治との一日
正在查询作品信息： 田崎草雲とその子
正在查询作品信息： 田沢稲船
正在查询作品信息： 他山の石
正在查询作品信息： だしの取り方
正在查询作品信息： 太十と其犬
正在查询作品信息： 太政官
正在查询作品信息： 多神教
正在查询作品信息： ダス・ゲマイネ
正在查询作品信息： たずねびと
正在查询作品信息： 黄昏
正在查询作品信息： 黄昏の告白
正在查询作品信息： 黄昏の地中海
正在查询作品信息： 黄昏の堤
正在查询作品信息： たたかいの中に
正在查询作品信息： 〔たゞかたくなのみをわぶる〕
正在查询作品信息： Tada-Dada of Alangri-Gloriban
正在查询作品信息： 忠直卿行状記
正在查询作品信息： たゞの文学
正在查询作品信息： 爛
正在查询作品信息： だだをこねる
正在查询作品信息： 起ち上る大阪
正在查询作品信息： 立枯れ
正在查询作品信息： 立すくむ
正在查询作品信息： 橘曙覧
正在查询作品信息： 橘曙覧評伝
正在查询作品信息： 立待岬にいたりて
正在查询作品信息： 立見の金網について
正在查询作品信息： 脱出と回帰
正在查询作品信息： 脱退問題是非
正在查询作品信息： 竜田丸の中毒事件
正在查询作品信息： ダッタン海峡
正在查询作品信息： 脱帽
正在查询作品信息： 

正在查询作品信息： 春がふたたび牢獄にやってきた！
正在查询作品信息： 春着
正在查询作品信息： 春はよみがえる
正在查询作品信息： ハルピンの一夜
正在查询作品信息： 春深く
正在查询作品信息： 春山秋山
正在查询作品信息： ハルレム
正在查询作品信息： 春六題
正在查询作品信息： 馬鈴薯階級の詩
正在查询作品信息： 馬鈴薯からトマト迄
正在查询作品信息： 馬鈴薯の花
正在查询作品信息： ハワイの食用蛙
正在查询作品信息： パンアテナイア祭の夢
正在查询作品信息： 晩夏
正在查询作品信息： 鷭狩
正在查询作品信息： 晩菊
正在查询作品信息： 反逆
正在查询作品信息： 反逆の呂律
正在查询作品信息： 反キリスト教運動
正在查询作品信息： 手巾
正在查询作品信息： 判決
正在查询作品信息： 反抗
正在查询作品信息： 瘢痕
正在查询作品信息： 晩餐
正在查询作品信息： 半七捕物帳
正在查询作品信息： マーカス・ショーとレビュー式教育
正在查询作品信息： 麻雀殺人事件
正在查询作品信息： 麻雀を語る
正在查询作品信息： マードック先生の『日本歴史』
正在查询作品信息： まあまあ居士の弁
正在查询作品信息： マイクロフォン
正在查询作品信息： マイクロフォン―雑感―
正在查询作品信息： マイクロフォン―八月増刊『陰獣』を中心にして―
正在查询作品信息： 迷子
正在查询作品信息： 迷子になつた上等兵（ラヂオドラマ）
正在查询作品信息： 迷子ノジヤガイモ
正在查询作品信息： 舞子より須磨へ
正在查询作品信息： 舞じたく
正在查询作品信息： 舞姫
正在查询作品信息： まえがき（『真実に生きた女性たち』）
正在查询作品信息： 魔王物語
正在查询作品信息： マカーガー峡谷の秘密
正在查询作品信息： まかないの棒
正在查询作品信息： 曲れる者
正在查询作品信息： マギーさんのオムレツ。
正在查询作品信息： 牧野さんの祭典によせて
正在查询作品信息： 牧野さんの死
正在查询作品信息： 牧野富太郎自叙伝
正在查询作品信息： 間木老人
正在查询作品信息： 幕間
正在查询作品信息： 幕が下りて
正在查询作品信息： マクシム・ゴーリキイについて
正在查询作品信息： マクシム・ゴーリキイによって描かれた婦人
正在查询作品信息： マクシム・ゴーリキイの伝記
正在查询作品信息： マクシム・ゴーリキイの

正在查询作品信息： 活人形
正在查询作品信息： 石狩川
正在查询作品信息： イシカリの川
正在查询作品信息： 石川五右衛門の生立
正在查询作品信息： 石川啄木と小奴
正在查询作品信息： 意識と時間との関係
正在查询作品信息： 石工
正在查询作品信息： 石ころ路
正在查询作品信息： 縊死体
正在查询作品信息： 医師高間房一氏
正在查询作品信息： 石段上りの街
正在查询作品信息： 石段に鉄管
正在查询作品信息： 異質触媒作用
正在查询作品信息： 医師と旅行鞄の話
正在查询作品信息： 石の思ひ
正在查询作品信息： 石の思い
正在查询作品信息： 石の信仰とさえの神と
正在查询作品信息： 石塀幽霊
正在查询作品信息： 医者と赤ン坊
正在查询作品信息： 医者と病人
正在查询作品信息： 医術の進歩
正在查询作品信息： 遺書
正在查询作品信息： 衣裳戸棚
正在查询作品信息： 衣食住雑感
正在查询作品信息： 遺書の一部より
正在查询作品信息： 石を愛するもの
正在查询作品信息： 石を食う
正在查询作品信息： 石をしょわずに
正在查询作品信息： 石を積む
正在查询作品信息： 石を投ぐるもの
正在查询作品信息： 石をのせた車
正在查询作品信息： 維新史の資料に就て
正在查询作品信息： いづこへ
正在查询作品信息： いずこへ
正在查询作品信息： 椅子と電車
正在查询作品信息： 伊豆の伊東へ
正在查询作品信息： 伊豆の国にて
正在查询作品信息： 伊豆の旅
正在查询作品信息： 泉
正在查询作品信息： 泉ある家
正在查询作品信息： 泉鏡花作『外科室』
正在查询作品信息： 泉鏡花先生のこと
正在查询作品信息： 泉山問題について
正在查询作品信息： イズムと云ふ語の意味次第
正在查询作品信息： イズムの功過
正在查询作品信息： 出雲鉄と安来節
正在查询作品信息： 異性間の友情と恋愛
正在查询作品信息： 異性に対する感覚を洗練せよ
正在查询作品信息： 公孫樹
正在查询作品信息： 一葉女史の「たけくらべ」を読みて
正在查询作品信息： 一葉の日記
正在查询作品信息： いちょうの葉
正在查询作品信息： いてふの実
正在查询作品信息： いちょうの実
正在查询作品信息： 一利己主義者と友人との対話
正在查询作品信息： 一れつ
正在查询作品信息： 一連の非プロレタリア的作品
正在查询作品信息： 一老人
正在查询作

...

正在查询作品信息： 黄金豹
正在查询作品信息： 黄金風景
正在查询作品信息： 黄金を浴びる女
正在查询作品信息： 王さまと靴屋
正在查询作品信息： 王さまの感心された話
正在查询作品信息： 汪士秀
正在查询作品信息： 欧洲紀行
正在查询作品信息： 奥州における御館藤原氏
正在查询作品信息： 『桜樹』の序
正在查询作品信息： 往生絵巻
正在查询作品信息： 王成
正在查询作品信息： 逢瀬までの。
正在查询作品信息： 黄村先生言行録
正在查询作品信息： 桜桃
正在查询作品信息： 「黄銅時代」創作メモ
正在查询作品信息： 黄銅時代の為
正在查询作品信息： 応仁の乱
正在查询作品信息： 応仁の乱に就て
正在查询作品信息： 「王の玄関」イエーツ戯曲
正在查询作品信息： 往復帖
正在查询作品信息： 欧米各国　政教日記
正在查询作品信息： 欧米料理と日本
正在查询作品信息： お馬がかへる
正在查询作品信息： 逢魔の刻
正在查询作品信息： 鸚鵡
正在查询作品信息： 鸚鵡蔵代首伝説
正在查询作品信息： 鸚鵡小町
正在查询作品信息： 鸚鵡のイズム
正在查询作品信息： 鸚鵡のゐる部屋
正在查询作品信息： 鸚鵡の思ひ出
正在查询作品信息： お江戸は火事だ
正在查询作品信息： 大井川奥山の話
正在查询作品信息： 大いなるもの
正在查询作品信息： 大井広介といふ男
正在查询作品信息： 大江戸黄金狂
正在查询作品信息： 大江満雄に
正在查询作品信息： 大江山
正在查询作品信息： 大岡越前
正在查询作品信息： 大岡越前の独立
正在查询作品信息： 巨男の話
正在查询作品信息： おおかみと七ひきのこどもやぎ
正在查询作品信息： おおかみと人
正在查询作品信息： 狼の怪
正在查询作品信息： おおかみをだましたおじいさん
正在查询作品信息： 大川の水
正在查询作品信息： 大川ばた
正在查询作品信息： 大きい足袋
正在查询作品信息： 大きなかしの木
正在查询作品信息： 大きなかに
正在查询作品信息： 大きな蝙蝠傘
正在查询作品信息： 大きな手
正在查询作品信息： 大きな怪物
正在查询作品信息： 大きな夢
正在查询作品信息： 大久保湖州
正在查询作品信息： Ｏ君の新秋
正在查询作品信息： 大阪といふところ
正在查询作品信息： 大阪夏之陣
正在查询作品信息： 大阪の朝
正在查询作品信息： 大阪の可能性
正在查询作

正在查询作品信息： 女の姿
正在查询作品信息： 女の手帖
正在查询作品信息： 女の出る蚊帳
正在查询作品信息： 女の話・花の話
正在查询作品信息： 女の膝
正在查询作品信息： 女の歴史
正在查询作品信息： 〔女は　ライラツクのにほひを好むと〕
正在查询作品信息： 女百貨店
正在查询作品信息： 「女らしさ」とは
正在查询作品信息： 「女らしさ」とは何か
正在查询作品信息： 「女らしさ」について
正在查询作品信息： 音波の殺人
正在查询作品信息： Ombra di Venezia
正在查询作品信息： 隠亡堀
正在查询作品信息： 御身
正在查询作品信息： 小浅間
正在查询作品信息： 小鰺釣
正在查询作品信息： 恋
正在查询作品信息： 鯉
正在查询作品信息： 恋衣
正在查询作品信息： 小石川台
正在查询作品信息： 恋しき最後の丘
正在查询作品信息： 碁石を呑んだ八っちゃん
正在查询作品信息： 小泉八雲秘稿画本「妖魔詩話」
正在查询作品信息： 恋妻であり敵であった
正在查询作品信息： 恋と十手と巾着切
正在查询作品信息： 小犬
正在查询作品信息： コイヌ
正在查询作品信息： 小犬と太郎さん
正在查询作品信息： こいぬ　の　ちびすけ
正在查询作品信息： 恋の一杯売
正在查询作品信息： 恋文
正在查询作品信息： 小祝の一家
正在查询作品信息： 恋を恋する人
正在查询作品信息： 恋をしに行く（「女体」につゞく）
正在查询作品信息： 好意
正在查询作品信息： 斯ういう気持
正在查询作品信息： こういう月評が欲しい
正在查询作品信息： 孔乙己
正在查询作品信息： 行為の価値
正在查询作品信息： 貢院の春
正在查询作品信息： 幸運の手紙のよりどころ
正在查询作品信息： 幸運の黒子
正在查询作品信息： 耕耘部の時計
正在查询作品信息： 行雲流水
正在查询作品信息： 黄英
正在查询作品信息： 公益に有害の鉱業を停止せざるの儀に付質問書
正在查询作品信息： 講演会　其他（大正十五年二月号）
正在查询作品信息： 講演軍記
正在查询作品信息： 公園の花と毒蛾
正在查询作品信息： 公園へ行く道
正在查询作品信息： 紅黄録
正在查询作品信息： 好悪の論
正在查询作品信息： 航海
正在查询作品信息： 郊外
正在查询作品信息： 号外
正在查询作品信息： 公開状
正在查询作品信息： 公開書架（Open-sh

正在查询作品信息： 碁にも名人戦つくれ
正在查询作品信息： 子猫
正在查询作品信息： 仔猫の裁判
正在查询作品信息： 仔猫の「トラ」
正在查询作品信息： 「子猫ノハナシ」
正在查询作品信息： 小ねこはなにを知ったか
正在查询作品信息： 五年後
正在查询作品信息： 近衛内閣の常識性
正在查询作品信息： この子
正在查询作品信息： 『この心の誇り』
正在查询作品信息： このごろ
正在查询作品信息： この頃
正在查询作品信息： この頃の皇太子殿下
正在查询作品信息： このごろの人気
正在查询作品信息： この子を残して
正在查询作品信息： この初冬
正在查询作品信息： このたび大阪
正在查询作品信息： 碁の手直り表
正在查询作品信息： この夏
正在查询作品信息： この握りめし
正在查询作品信息： 『この果てに君ある如く』の選後に
正在查询作品信息： 木の葉山女魚
正在查询作品信息： 〔このみちの醸すがごとく〕
正在查询作品信息： この三つのことば
正在查询作品信息： 古盃
正在查询作品信息： 琥珀揚げ
正在查询作品信息： 琥珀のパイプ
正在查询作品信息： 小林虐殺一周忌　二月二十日
正在查询作品信息： 小林さんと私のツキアイ
正在查询作品信息： 小林多喜二のお母さん
正在查询作品信息： 小林多喜二の今日における意義
正在查询作品信息： 〔小林秀雄氏への公開状〕
正在查询作品信息： 小林秀雄小論
正在查询作品信息： 小春
正在查询作品信息： 小春の狐
正在查询作品信息： 湖畔手記
正在查询作品信息： 湖畔亭事件
正在查询作品信息： コピーライトの史的展開
正在查询作品信息： 小日向台
正在查询作品信息： 五百句
正在查询作品信息： 五百五十句
正在查询作品信息： 瘤
正在查询作品信息： 鼓舞さるべき仕事
正在查询作品信息： 辛夷の花
正在查询作品信息： 瘤とり
正在查询作品信息： ゴボウ君と大根君
正在查询作品信息： コポオの弟子たち
正在查询作品信息： 古木
正在查询作品信息： 五本の指
正在查询作品信息： こま
正在查询作品信息： 独楽
正在查询作品信息： こま犬
正在查询作品信息： 駒台の発案者
正在查询作品信息： 小町の芍薬
正在查询作品信息： 駒鳥の胸
正在查询作品信息： 駒のいななき
正在查询作品信息： 五味氏の宝物
正在查询作品信息： 小むすめ
正在查询作品信息

正在查询作品信息： 野路の菊
正在查询作品信息： 野宿
正在查询作品信息： のぞき眼鏡
正在查询作品信息： 帆
正在查询作品信息： 宝永噴火
正在查询作品信息： 法悦クラブ
正在查询作品信息： 放翁鑑賞
正在查询作品信息： 法王の祈祷
正在查询作品信息： 忘恩
正在查询作品信息： 報恩記
正在查询作品信息： 望岳都東京
正在查询作品信息： 法学とは何か
正在查询作品信息： 防火栓
正在查询作品信息： 幇間
正在查询作品信息： 判官三郎の正体
正在查询作品信息： 傍観者の言
正在查询作品信息： 防寒戸
正在查询作品信息： ほうき一本
正在查询作品信息： 彗星の話
正在查询作品信息： 望郷
正在查询作品信息： 奉教人の死
正在查询作品信息： 方言
正在查询作品信息： 冒険
正在查询作品信息： 方言について
正在查询作品信息： 亡国に至るを知らざれば之即ち亡国の儀に付質問
正在查询作品信息： 封三娘
正在查询作品信息： 帽子のない水兵
正在查询作品信息： バウシ　ノ　ユクヘ
正在查询作品信息： 報酬
正在查询作品信息： 房州紀行
正在查询作品信息： 房州の一夏
正在查询作品信息： 忘春詩集
正在查询作品信息： 方丈記
正在查询作品信息： 放生津物語
正在查询作品信息： 北條より一ノ宮へ
正在查询作品信息： 放心教授
正在查询作品信息： 傍人の言
正在查询作品信息： 放水路
正在查询作品信息： 宝石商
正在查询作品信息： 宝石の序曲
正在查询作品信息： 防雪林
正在查询作品信息： 放送された遺言
正在查询作品信息： 宝蔵の短刀
正在查询作品信息： 法窓夜話
正在查询作品信息： 方則について
正在查询作品信息： 亡弟
正在查询作品信息： 放蕩の虫
正在查询作品信息： ほうとする話
正在查询作品信息： 法然行伝
正在查询作品信息： 忙ノ説
正在查询作品信息： ポウの本質
正在查询作品信息： 防備隊
正在查询作品信息： 暴風雨に終わった一日
正在查询作品信息： 暴風への郷愁
正在查询作品信息： 放免考
正在查询作品信息： 葬られたる秘密
正在查询作品信息： 泡鳴五部作
正在查询作品信息： 坊や
正在查询作品信息： 鳳来寺紀行
正在查询作品信息： 「ほ」・「うら」から「ほがひ」へ
正在查询作品信息： 法律的独立人格の承認
正在查询作品信息： 法隆寺再建非再建論の回顧
正在查询作品信

正在查询作品信息： 余裕のことなど
正在查询作品信息： 漂著石神論計画
正在查询作品信息： 頼朝の最後
正在查询作品信息： 夜
正在查询作品信息： 夜の赤坂
正在查询作品信息： 夜のガスパール　抄
正在查询作品信息： 夜の奇蹟
正在查询作品信息： 夜の靴
正在查询作品信息： 夜の構図
正在查询作品信息： 夜の讃歌
正在查询作品信息： 夜の三角形
正在查询作品信息： 夜の進軍らっぱ
正在查询作品信息： 夜の隅田川
正在查询作品信息： 夜の鳥
正在查询作品信息： 夜の浪
正在查询作品信息： 夜の道づれ
正在查询作品信息： 夜の喜び
正在查询作品信息： 夜の若葉
正在查询作品信息： 夜福
正在查询作品信息： 余齢初旅
正在查询作品信息： 鎧の挿話
正在查询作品信息： 余録（一九二四年より）
正在查询作品信息： 悦しき知識
正在查询作品信息： 老嫗面
正在查询作品信息： 狼園
正在查询作品信息： 老猾抄
正在查询作品信息： 老妓抄
正在查询作品信息： 老境
正在查询作品信息： 老境なるかな
正在查询作品信息： 老犬の怪
正在查询作品信息： 牢獄の花嫁
正在查询作品信息： 牢獄の半日
正在查询作品信息： 老狐の怪
正在查询作品信息： 老残
正在查询作品信息： 老子化胡経
正在查询作品信息： 狼疾記
正在查询作品信息： 老獣医
正在查询作品信息： 老中の眼鏡
正在查询作品信息： 老主の一時期
正在查询作品信息： 老巡査
正在查询作品信息： 老将軍と大学教授
正在查询作品信息： 老人
正在查询作品信息： 老人退場説
正在查询作品信息： 老人と海
正在查询作品信息： 濫僧考
正在查询作品信息： ろうそくと貝がら
正在查询作品信息： 労働祭歌（Ⅰ）
正在查询作品信息： 労働者大会
正在查询作品信息： 労働者の居ない船
正在查询作品信息： 労働者農民の国家とブルジョア地主の国家
正在查询作品信息： 『労働戦線』小説選後評
正在查询作品信息： 労働の精神
正在查询作品信息： 老年
正在查询作品信息： 老年と人生
正在查询作品信息： 老婆
正在查询作品信息： 臘梅
正在查询作品信息： 老病について
正在查询作品信息： 老夫婦
正在查询作品信息： 浪曼的月評
正在查询作品信息： 浪曼的時評
正在查询作品信息： ロウモン街の自殺ホテル
正在查询作品信息： 老狸伝
正在查询作品信息： ローマ字論

In [28]:
get_sakuhinn()

正在查询作品列表


[(1,
  'あ.a',
  'ああ華族様だよ\u3000と私は嘘を吐くのであった',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000020/card2569.html'),
 (2,
  'あ.a',
  'ア、秋',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000035/card236.html'),
 (3,
  'あ.a',
  'ああ玉杯に花うけて',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000575/card3585.html'),
 (4,
  'あ.a',
  'アーサー王物語',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000140/card50131.html'),
 (5,
  'あ.a',
  '「ああしんど」',
  'Unknown',
  'http://www.aozora.gr.jp/cards/001351/card49202.html'),
 (6,
  'あ.a',
  'ああ東京は食い倒れ',
  'Unknown',
  'http://www.aozora.gr.jp/cards/001558/card52318.html'),
 (7,
  'あ.a',
  'アーニイ・パイルの前に立ちて',
  'Unknown',
  'http://www.aozora.gr.jp/cards/001256/card46657.html'),
 (8,
  'あ.a',
  'あゝ二十年',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000355/card47298.html'),
 (9,
  'あ.a',
  'Ｒ漁場と都の酒場で',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000183/card45373.html'),
 (10,
  'あ.a',
  '愛',
  'Unknown',
  'http://www.aozora.gr.jp/cards/000311/card4027.html'),
 (11,

#### bulid_db()
#### get_sakusya()
#### get_sakuhinn()
#### sakusyainsert(_sakusyacatalog,_sakusyaname,_sakuhinn,_sakusyalink)
#### sakuhinninsert(_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuyhinnlink)
#### get_onesakuhinn(_name)
#### get_onesakusya(_name)
#### get_kanasakuhinn(_sakuhinncatalog)
#### get_kanasakusya(_sakusyacatalog)

# 获取作者分类链接和作品分类链接

In [ ]:
def get_catalog_links():
    """返回作者分类目录和作品分类目录的标签项（每一项的["href"]获取网址后半段，需在前加homepage，每项的.text获取名称）"""
    homepage = 'http://www.aozora.gr.jp/'
    reg_homepage = requests.get(homepage,headers=headers)
    soup_homepage = bs(reg_homepage.content.decode("utf-8"),'lxml')
    links = soup_homepage.findAll(name='a',attrs={"href":re.compile(r'^index')})
    sakusyas = links[1:12]
    sakuhins = links[12:-3]
    #print(sakuhins[0]['href'])
    return sakusyas,sakuhins

## 获取并储存类别中作家作品链接

In [ ]:
def save_sakuhinn(_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuyhinnlink):
    try:
        if not get_onesakuhinn(_sakuhinnname):
            sakuhinninsert(_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuyhinnlink)
            print("已储存作品："+_sakuhinnname)
        else:
            print("该作品已存在： "+_sakuhinnname)
    except:
        pass
def save_sakusya(_sakusyacatalog,_sakusyaname,_sakuhinn,_sakusyalink):
    try:
        if not get_onesakusya(_sakusyaname):
            sakusyainsert(_sakusyacatalog,_sakusyaname,_sakuhinn,_sakusyalink)
            print("已储存作家："+_sakusyaname)
        else:
            print("该作家已存在： "+_sakusyaname)
    except:
        pass
        
def get_saka_links(catalog,url):
    """输入作者类别链接，返回该类作品分类列表链接标签，（每一项的["href"]获取网址后半段，需在前加homepage，每项的.text获取名称）"""
    a = url[-7:-5]
    if '_' in a:
        p = a[1]
    else:
        p = a[0:2]
    url = "http://www.aozora.gr.jp/index_pages/person_"+p+".html"
    reg_saka = requests.get(url,headers=headers)
    soup_saka = bs(reg_saka.content.decode("utf-8"),'lxml')
    links = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^person')})
    l = links[:-10]
    for i in range(len(l)):
        l[i] = {'catalog':catalog + '.'+ p,'sakusyaname':l[i].text,'link':'http://www.aozora.gr.jp/'+l[i]['href']}
        o = l[i]
        # save_sakusya(_sakusyacatalog,_sakusyaname,_sakuhinn,_sakusyalink)
        save_sakusya(o['catalog'],o['sakusyaname'],'Unknown',o['link'])

def flatten(a):
    """列表降维"""
    for each in a:
        if not isinstance(each, list):
            yield each
        else:
            yield from flatten(each)
            
def get_sakuhin_links(catalog,url):
    """获取假名作品排序列表"""
    a = url[-8:-5]
    if '_' in a:
        p = a[1]
    else:
        p = a[0:2]
    reg_saka = requests.get(url,headers=headers)
    soup_saka = bs(reg_saka.content.decode("utf-8"),'lxml')
    links = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^../card')})
    link = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^sakuhin')})
    try:
        for j in range(2,int(link[-2].text)+1):# 本该加1
            l = list(flatten(links))
            for i in range(len(l)):
                l[i] = {'catalog':catalog + '.'+ p,'sakuhinnname':l[i].text,'link':'http://www.aozora.gr.jp/'+l[i]['href']}
                o = l[i]
                # save_sakuhinn(_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuyhinnlink):
                save_sakuhinn(o['catalog'],o['sakuhinnname'],'Unknown',o['link'])

            print("Getting page of "+p+str(j))
            url = "http://www.aozora.gr.jp/index_pages/sakuhin_"+ p + str(j) +".html"
            reg_saka = requests.get(url,headers=headers)
            soup_saka = bs(reg_saka.content.decode("utf-8"),'lxml')
            links = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^../card')})
    except:
        for j in range(2,4):# 本该加1
            l = list(flatten(links))
            for i in range(len(l)):
                l[i] = {'catalog':catalog + '.'+ p,'sakuhinnname':l[i].text,'link':'http://www.aozora.gr.jp/'+l[i]['href']}
                o = l[i]
                # save_sakuhinn(_sakuhinncatalog,_sakuhinnname,_sakusya,_sakuyhinnlink):
                save_sakuhinn(o['catalog'],o['sakuhinnname'],'Unknown',o['link'])

            print("Getting page of "+p+str(j))
            url = "http://www.aozora.gr.jp/index_pages/sakuhin_"+ p + str(j) +".html"
            reg_saka = requests.get(url,headers=headers)
            soup_saka = bs(reg_saka.content.decode("utf-8"),'lxml')
            links = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^../card')})
        
        


def main():
    import time
    sakusyas,sakuhins = get_catalog_links()
    print("You have links about "+str(len(sakuhins)))
    for i in range(len(sakuhins)):
        print("Getting link "+str(i))
        sakuhins[i] =  {'catalog':sakuhins[i].text,'link':'http://www.aozora.gr.jp/' + sakuhins[i]['href']}
    for i in range(len(sakusyas)):
        print("Getting link "+str(i))
        sakusyas[i] = {'catalog':sakusyas[i].text,'link':'http://www.aozora.gr.jp/' + sakusyas[i]['href']}
    #sakusyas = sakusyas[1:]
    #for i in sakusyas:
        #print("开始时间： "+str(time.localtime(time.time())))
        #get_saka_links(i['catalog'],i['link'])
        #print("结束时间： "+str(time.localtime(time.time())))
    sakuhins = sakuhins[18:]
    for i in sakuhins:
        print("开始时间： "+str(time.localtime(time.time())))
        get_sakuhin_links(i['catalog'],i['link'])
        print("结束时间： "+str(time.localtime(time.time())))

In [123]:
print(len(get_sakuhinn()))

正在查询作品列表
2885


## 保存列表

In [ ]:
import csv
def save_data(p,datas):
    with open(p + '.csv', 'at')as f:
        P_csv = csv.writer(f)
        P_csv.writerows(datas)
    return true


In [86]:
links = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^sakuhin')})
for i in range(1:int(links[-2].text)):
    url = "http://www.aozora.gr.jp/index_pages/sakuhin_a1.html"
    reg_saka = requests.get(url,headers=headers)
    soup_saka = bs(reg_saka.content.decode("utf-8"),'lxml')
    l = soup_saka.findAll(name='a',attrs={"href":re.compile(r'^../card')})
    links.append(l)
print()

17


# 获取作者作品信息

In [4]:
def get_info(url):
    info = []
    return info
    

# 获取作品

In [51]:
def get_work(url):
    #url = 'http://www.aozora.gr.jp/cards/001540/card53885.html'
    q = 'card'+url.split('card')[-1]
    reg_work_homepage = requests.get(url,headers=headers)
    soup_work_homepage = bs(reg_work_homepage.content.decode("utf-8"),'lxml')
    links = soup_work_homepage.findAll(name='a',attrs={"href":re.compile(r'^./files/')})[-1]['href']
    link = url.replace(q,'')+links[2:]
    return link

In [48]:
url = 'http://www.aozora.gr.jp/cards/001529/card44909.html'
q = 'card'+url.split('card')[-1]
reg_work_homepage = requests.get(url,headers=headers)
soup_work_homepage = bs(reg_work_homepage.content.decode("utf-8"),'lxml')
links = soup_work_homepage.findAll(name='a',attrs={"href":re.compile(r'^./files/')})[-1]['href']
link = url.replace(q,'')+links[2:]

In [53]:
print(get_work('http://www.aozora.gr.jp/cards/001529/card44909.html'))

http://www.aozora.gr.jp/cards/001529/files/44909_29558.html
